In [1]:
!pip install farm-haystack
!pip install urllib3==1.25.4
!!pip install sentence-transformers
!pip install faiss-cpu
!pip install --force-reinstall -v "SQLAlchemy==1.4.47"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.9/789.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13

In [2]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever
from haystack import Document
import pandas as pd
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser
from haystack.pipelines import Pipeline
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
DEFAULT_EMBEDDING_MODEL= "sentence-transformers/multi-qa-mpnet-base-dot-v1"
DEFAULT_READER = "deepset/roberta-base-squad2"
DEFAULT_GENERATOR = "pszemraj/flan-t5-large-instruct-dolly_hhrlhf"
DATA_FILE= "AIUseCases.csv"
PARENT_DIR= "/content/gdrive/MyDrive/"

In [4]:
masterAI_df= pd.read_csv(PARENT_DIR + DATA_FILE)
masterAI_df= masterAI_df.drop(columns=['Unnamed: 0'])
masterAI_df.head()

,Use Case Name,Summary of Use Case,Agency,Bureau / Department,Use Case ID,Stage of System Development Life Cycle,Date Initiated (if applicable),Date when Development and/or Acquisition began (if applicable),Date Implemented (if applicable),Developer Information,...,Additional Life Cycle Comments,Agency Training Used,Included In Enterprise Inventory,Public Data Link,Code Included In Agency Inventory,FISMA System Name,Withhold Use Case,Explanation To Withhold Use Case,Releaseable to the Public,text
0,4% Repair Dashboard,The model reviews the descriptions of expenses...,United States Department of Agriculture,ARS,USDA-1,Operation and Management,0,0,0,In-house,...,0,0,0,0,0,0,0,0,0,"United States Department of Agriculture, 4% Re..."
1,ARS Project Mapping,NLP of research project plans including term a...,United States Department of Agriculture,ARS,USDA-2,Operation and Management,1/1/2020,1/1/2021,5/1/2022,Contracted,...,0,0,0,0,0,0,0,0,0,"United States Department of Agriculture, ARS P..."
2,NAL Automated indexing,"Cogito (vendor) software, uses AI for automate...",United States Department of Agriculture,ARS,USDA-3,Operation and Management,6/1/2011,6/1/2012,12/1/2012,In-house,...,0,0,0,0,0,0,0,0,0,"United States Department of Agriculture, NAL A..."
3,Predictive modeling of invasive pest species a...,Macine learning algorithms are used to develop...,United States Department of Agriculture,APHIS,USDA-4,Operation and Management,0,0,0,In-house,...,0,0,0,0,0,0,0,0,0,"United States Department of Agriculture, Predi..."
4,Detection of pre-symptomatic HLB infected citrus,Identify pixels with HLB infection signature i...,United States Department of Agriculture,APHIS,USDA-5,Operation and Management,0,0,0,In-house,...,0,0,0,0,0,0,0,0,0,"United States Department of Agriculture, Detec..."


In [5]:
def load_document_store():
    return FAISSDocumentStore(faiss_index_factory_str="Flat",return_embedding=True)

In [6]:
def get_embedding_retriever(doc_store):
    return EmbeddingRetriever(
        document_store=doc_store,
        embedding_model=DEFAULT_EMBEDDING_MODEL,
    )

In [7]:
def create_document():
    table_df = pd.read_csv(PARENT_DIR + DATA_FILE)
    table_dict= table_df.to_dict(orient='records')
    return [Document(content = item['text'], meta = item) for item in table_dict]

In [8]:
def get_reader():
    return FARMReader(model_name_or_path=DEFAULT_READER)

In [9]:
def summarization_template():
    return 'Summarize this document: {join(documents)}\nSummary:'

In [10]:
def question_answering_template():
    return 'Given the context please answer the question. Context: {join(documents)};\nQuestion: {query};\nAnswer:'

In [11]:
def get_prompt_node():
    rag_prompt = PromptTemplate(
        prompt=summarization_template(),
        output_parser=AnswerParser()
    )
    return PromptNode(model_name_or_path=DEFAULT_GENERATOR, default_prompt_template=rag_prompt)

In [12]:
def QADocStoreRetriever():
    document_store_faiss= load_document_store()
    retriever_faiss= get_embedding_retriever(document_store_faiss)
    doc= create_document()
    document_store_faiss.delete_documents()
    document_store_faiss.write_documents(doc)
    document_store_faiss.update_embeddings(retriever=retriever_faiss)
    return retriever_faiss

In [13]:
def extractiveQA(query):
    reader= get_reader()
    pipeline= ExtractiveQAPipeline(reader, retriever_faiss)
    prediction = pipeline.run(
      query=query,
      params={
          "Retriever": {"top_k": 10},
          "Reader": {"top_k": 5}
        }
      )
    return prediction["answers"][0].answer

In [14]:
def generativeQA(query):
    prompt_node= get_prompt_node()
    pipeline= Pipeline()
    pipeline.add_node(component=retriever_faiss, name="retriever", inputs=["Query"])
    pipeline.add_node(component=prompt_node, name="prompt_node", inputs=["retriever"])
    output = pipeline.run(query=query)
    return output["answers"][0].answer


In [15]:
retriever_faiss= QADocStoreRetriever()

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Writing Documents: 10000it [00:17, 588.17it/s]
Updating Embedding:   0%|          | 0/142 [00:00<?, ? docs/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Documents Processed: 10000 docs [00:12, 770.45 docs/s]


In [16]:
extractiveQA("Which agencies and bureaus are using deepfake detection?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.67 Batches/s]


'Department of the Interior, TMDL and Data Mining Investigations'

In [17]:
generativeQA("Which agencies and bureaus are using deepfake detection?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1554 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `1.3` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(
Both `max_new_tokens` (=192) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformer

'The Department of the Interior is developing a new system to help identify corrosion levels in wells. This system will be used to identify areas of concern and provide recommendations for further inspections.'